In [ ]:
!pip install lightautoml

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score
from scipy.stats import mode

In [ ]:
N_THREADS = 4 # threads cnt for lgbm and linear models
N_FOLDS = 3 # folds cnt for AutoML
RANDOM_STATE = 42 # fixed random state for various reasons
TEST_SIZE = 0.3 # Test size for metric check
TIMEOUT = 300 # Time in seconds for automl run


TARGET_NAME = 'Survived' # Target column name
# TARGET_NAME = 'TARGET' # Target column name

TARGET_NAME

In [ ]:
train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')
ss = pd.read_csv('../input/titanic/gender_submission.csv')


# train = pd.read_csv('../input/santander-customer-satisfaction/train.csv')
# test = pd.read_csv('../input/santander-customer-satisfaction/test.csv')
# ss = pd.read_csv('../input/santander-customer-satisfaction/sample_submission.csv')

# for_drop = [col for col in train.columns if col.startswith('town_emb')] + ['id']
# train.drop(for_drop, axis=1, inplace=True)

In [ ]:
train

In [ ]:
train_data, test_data = train_test_split(train, 
                                         test_size=TEST_SIZE, 
                                         stratify=train[TARGET_NAME], 
                                         random_state=RANDOM_STATE)
print('Data splitted. Parts sizes: train_data = {}, test_data = {}'
              .format(train_data.shape, test_data.shape))

In [ ]:
result = pd.DataFrame(columns=['TEST score'])

In [ ]:
# roles = {'target': TARGET_NAME, 'category': ['profile_town', 'profile_district', 'engine_type', 'recommended_policy_type', 'car_color', 'car_type']}
roles = {'target': TARGET_NAME, 
         'drop': ['PassengerId', ],
        }

task = Task('binary', )
reader = PandasToPandasReader(task, cv=N_FOLDS, random_state=RANDOM_STATE, advanced_roles=False)

# model0 = BoostLGBM(default_params={'learning_rate': 0.1, 'num_leaves': 64, 'seed': 42, 'num_threads': N_THREADS})
# pie = ModelBasedImportanceEstimator()
# selector = ImportanceCutoffSelector(LGBSimpleFeatures(), model0, pie, cutoff=-9999)

# model1 = BoostLGBM(default_params={'learning_rate': 0.1, 'num_leaves': 64, 'seed': 42, 'num_threads': N_THREADS})
# pie1 = NpPermutationImportanceEstimator()
# selector1 = ImportanceCutoffSelector(EmptyFeaturePipeline(), model1, pie1, cutoff=-9999)

# pipe = LGBSimplerFeatures(selector, top_category=4).append(GroupByPipeline(None, top_category=4, top_numeric=4, check_mode=True, verbose_mode=True))
pipe = GroupByPipeline(None, top_category=4, top_numeric=4, check_mode=True, verbose_mode=True)

model = BoostLGBM(default_params={'learning_rate': 0.05, 'num_leaves': 128, 'seed': 1, 'num_threads': N_THREADS})
pipeline = MLPipeline([(model),], 
#                       pre_selection=selector, 
                      features_pipeline=pipe, 
#                       post_selection=selector1
                     )

automl = AutoML(reader, [
    [pipeline],
], skip_conn=False, verbose=1)

oof_pred = automl.fit_predict(train_data, roles = roles)

test_pred = automl.predict(test_data)

print('Check scores...')
print('OOF score: {}'.format(roc_auc_score(train_data[TARGET_NAME].values,
                                           oof_pred.data[:, 0])))
test_automl = roc_auc_score(test_data[TARGET_NAME].values, test_pred.data[:, 0])
print('TEST score: {}'.format(test_automl))
print('')
print(pie1.get_features_score())

In [ ]:
from copy import deepcopy

# Imports from our package
from lightautoml.automl.base import AutoML
from lightautoml.tasks import Task

from lightautoml.dataset.roles import DatetimeRole, NumericRole, CategoryRole
from lightautoml.dataset.np_pd_dataset import NumpyDataset, PandasDataset

from lightautoml.pipelines.utils import get_columns_by_role
from lightautoml.pipelines.features.base import EmptyFeaturePipeline, FeaturesPipeline, TabularDataFeatures
from lightautoml.pipelines.features.base import FeaturesPipeline, TabularDataFeatures

from lightautoml.pipelines.features.lgb_pipeline import LGBAdvancedPipeline, LGBSimpleFeatures
from lightautoml.pipelines.ml.base import MLPipeline

from lightautoml.pipelines.selection.base import  SelectionPipeline, ImportanceEstimator, PredefinedSelector
from lightautoml.pipelines.selection.importance_based import ImportanceCutoffSelector, ModelBasedImportanceEstimator
from lightautoml.pipelines.selection.permutation_importance_based import NpPermutationImportanceEstimator, \
    NpIterativeFeatureSelector, _create_chunks_from_list

from lightautoml.ml_algo.boost_lgbm import BoostLGBM
from lightautoml.ml_algo.utils import tune_and_fit_predict


from lightautoml.transformers.base import LAMLTransformer, SequentialTransformer, UnionTransformer, ColumnsSelector, \
    ConvertDataset, ChangeRoles
from lightautoml.transformers.numeric import FillnaMedian
from lightautoml.transformers.categorical import LabelEncoder, OrdinalEncoder, CatIntersectstions, TargetEncoder
from lightautoml.transformers.datetime import TimeToNum

from lightautoml.reader.base import PandasToPandasReader


In [ ]:
class LGBSimplerFeatures(FeaturesPipeline, TabularDataFeatures):
    """Creates simple pipeline for tree based models.
    Simple but is ok for select features.
    Numeric stay as is, Datetime transforms to numeric.
    Categorical label encoding.
    Maps input to output features exactly one-to-one.
    """
    def __init__(self, feats_imp = None, top_category: int = 3, top_numeric: int = 3, to_TE=True, **kwargs):
        """

        """
        super().__init__(feats_imp=feats_imp)
        self.top_category = top_category
        self.top_numeric = top_numeric
        self.to_TE = to_TE

    def create_pipeline(self, train):
        """Create tree pipeline.
        Args:
            train: Dataset with train features.
        Returns:
            Composite datetime, categorical, numeric transformer.
        """
        
        # TODO: Transformer params to config
        transformers_list = []

        # process categories
        categories = get_columns_by_role(train, 'Category')
        cat_feats_to_select = []
        if len(categories) > self.top_category:
            cat_feats_to_select = self.get_top_categories(train, self.top_category)
        elif len(categories) > 0:
            cat_feats_to_select = categories
            
        if len(categories) > 0:
            cat_processing = SequentialTransformer([

                ColumnsSelector(keys=categories),
                LabelEncoder(subs=None, random_state=42),

            ])
            transformers_list.append(cat_processing)
            
        if len(categories) > 0:
            trns = [
                ColumnsSelector(keys=cat_feats_to_select),
                CatIntersectstions(subs=None, random_state=42)]
            if self.to_TE:
                trns += [TargetEncoder()]
            else:
                trns += [ChangeRoles(CategoryRole(np.float32))]
                
            cat_inter = SequentialTransformer(trns)
            transformers_list.append(cat_inter)   
        

        # process datetimes
        datetimes = get_columns_by_role(train, 'Datetime')
        if len(datetimes) > 0:
            dt_processing = SequentialTransformer([

                ColumnsSelector(keys=datetimes),
                TimeToNum()

            ])
            transformers_list.append(dt_processing)

        # process numbers
        #"""
        numerics = get_columns_by_role(train, 'Numeric')
        if len(numerics) > 0:
            num_processing = SequentialTransformer([

                ColumnsSelector(keys=numerics),
                ConvertDataset(dataset_type=NumpyDataset)

            ])
            transformers_list.append(num_processing)
        #"""
        union_all = UnionTransformer(transformers_list)

        return union_all

In [ ]:
# roles = {'target': TARGET_NAME, 'category': ['profile_town', 'profile_district', 'engine_type', 'recommended_policy_type', 'car_color', 'car_type']}
roles = {'target': TARGET_NAME, 
         'category': ['Name', 'Embarked', 'Sex', 'Ticket'],
         'drop': ['PassengerId', ],
        }

task = Task('binary', )
reader = PandasToPandasReader(task, cv=N_FOLDS, random_state=RANDOM_STATE, advanced_roles=False)

model0 = BoostLGBM(default_params={'learning_rate': 0.1, 'num_leaves': 64, 'seed': 42, 'num_threads': N_THREADS})
pie = ModelBasedImportanceEstimator()
selector = ImportanceCutoffSelector(LGBSimpleFeatures(), model0, pie, cutoff=-9999)

model1 = BoostLGBM(default_params={'learning_rate': 0.1, 'num_leaves': 64, 'seed': 42, 'num_threads': N_THREADS})
pie1 = NpPermutationImportanceEstimator()
selector1 = ImportanceCutoffSelector(EmptyFeaturePipeline(), model1, pie1, cutoff=-9999)

pipe = LGBSimpleFeatures()
model = BoostLGBM(default_params={'learning_rate': 0.05, 'num_leaves': 128, 'seed': 1, 'num_threads': N_THREADS})
pipeline = MLPipeline([(model),], pre_selection=selector, features_pipeline=pipe, post_selection=selector1)

automl = AutoML(reader, [
    [pipeline],
], skip_conn=False, verbose=0)

oof_pred = automl.fit_predict(train_data, roles = roles)

test_pred = automl.predict(test_data)

print('Check scores...')
print('OOF score: {}'.format(roc_auc_score(train_data[TARGET_NAME].values,
                                           oof_pred.data[:, 0])))
test_automl = roc_auc_score(test_data[TARGET_NAME].values, test_pred.data[:, 0])
print('TEST score: {}'.format(test_automl))
print('')
print(pie1.get_features_score())

In [ ]:
result.loc['simple pipeline, no TE', 'TEST score'] = np.round(test_automl, 4)
result

In [ ]:
# roles = {'target': TARGET_NAME, 'category': ['profile_town', 'profile_district', 'engine_type', 'recommended_policy_type', 'car_color', 'car_type']}
roles = {'target': TARGET_NAME, 
         'category': ['Name', 'Embarked', 'Sex', 'Ticket'],
         'drop': ['PassengerId', ],
        }

task = Task('binary', )
reader = PandasToPandasReader(task, cv=N_FOLDS, random_state=RANDOM_STATE, advanced_roles=False)

model0 = BoostLGBM(default_params={'learning_rate': 0.1, 'num_leaves': 64, 'seed': 42, 'num_threads': N_THREADS})
pie = ModelBasedImportanceEstimator()
selector = ImportanceCutoffSelector(LGBSimpleFeatures(), model0, pie, cutoff=-9999)

model1 = BoostLGBM(default_params={'learning_rate': 0.1, 'num_leaves': 64, 'seed': 42, 'num_threads': N_THREADS})
pie1 = NpPermutationImportanceEstimator()
selector1 = ImportanceCutoffSelector(EmptyFeaturePipeline(), model1, pie1, cutoff=-9999)

pipe = LGBSimplerFeatures(selector, top_category=2, to_TE=False)
model = BoostLGBM(default_params={'learning_rate': 0.05, 'num_leaves': 128, 'seed': 1, 'num_threads': N_THREADS})
pipeline = MLPipeline([(model),], pre_selection=selector, features_pipeline=pipe, post_selection=selector1)

automl = AutoML(reader, [
    [pipeline],
], skip_conn=False, verbose=0)

oof_pred = automl.fit_predict(train_data, roles = roles)

test_pred = automl.predict(test_data)

print('Check scores...')
print('OOF score: {}'.format(roc_auc_score(train_data[TARGET_NAME].values,
                                           oof_pred.data[:, 0])))
test_automl = roc_auc_score(test_data[TARGET_NAME].values, test_pred.data[:, 0])
print('TEST score: {}'.format(test_automl))
print('')
print(pie1.get_features_score())

In [ ]:
result.loc['simpler pipeline, 2 interactions, no TE', 'TEST score'] = np.round(test_automl, 4)
result

In [ ]:
# roles = {'target': TARGET_NAME, 'category': ['profile_town', 'profile_district', 'engine_type', 'recommended_policy_type', 'car_color', 'car_type']}
roles = {'target': TARGET_NAME, 
         'category': ['Name', 'Embarked', 'Sex', 'Ticket'],
         'drop': ['PassengerId', ],
        }

task = Task('binary', )
reader = PandasToPandasReader(task, cv=N_FOLDS, random_state=RANDOM_STATE, advanced_roles=False)

model0 = BoostLGBM(default_params={'learning_rate': 0.1, 'num_leaves': 64, 'seed': 42, 'num_threads': N_THREADS})
pie = ModelBasedImportanceEstimator()
selector = ImportanceCutoffSelector(LGBSimpleFeatures(), model0, pie, cutoff=-9999)

model1 = BoostLGBM(default_params={'learning_rate': 0.1, 'num_leaves': 64, 'seed': 42, 'num_threads': N_THREADS})
pie1 = NpPermutationImportanceEstimator()
selector1 = ImportanceCutoffSelector(EmptyFeaturePipeline(), model1, pie1, cutoff=-9999)

pipe = LGBSimplerFeatures(selector, top_category=4)
model = BoostLGBM(default_params={'learning_rate': 0.05, 'num_leaves': 128, 'seed': 1, 'num_threads': N_THREADS})
pipeline = MLPipeline([(model),], pre_selection=selector, features_pipeline=pipe, post_selection=selector1)

automl = AutoML(reader, [
    [pipeline],
], skip_conn=False, verbose=0)

oof_pred = automl.fit_predict(train_data, roles = roles)

test_pred = automl.predict(test_data)

print('Check scores...')
print('OOF score: {}'.format(roc_auc_score(train_data[TARGET_NAME].values,
                                           oof_pred.data[:, 0])))
test_automl = roc_auc_score(test_data[TARGET_NAME].values, test_pred.data[:, 0])
print('TEST score: {}'.format(test_automl))
print('')
print(pie1.get_features_score())

In [ ]:
result.loc['simpler pipeline, TE', 'TEST score'] = np.round(test_automl, 4)
result

In [ ]:
class GroupByTransformer(LAMLTransformer):
    _fit_checks = ()
    _transform_checks = ()
    _fname_prefix = 'grb'

    @property
    def features(self):
        """Features list."""

        return self._features

    def __init__(self, **kwargs):
        super().__init__()
        
        self.dicts = {}        

        self.check_mode = kwargs['check_mode'] if 'check_mode' in kwargs else False
        self.verbose_mode = kwargs['verbose_mode'] if 'verbose_mode' in kwargs else False

    def __get_mode(self, x):
        return mode(x)[0][0]

    def fit(self, dataset):    
        """fit"""

        if self.verbose_mode: print('GroupByTransformer.__fit_new')
        if self.verbose_mode: print('GroupByTransformer.__fit_new.type(dataset.data):', type(dataset.data.to_numpy()))

        # set transformer names and add checks
        for check_func in self._fit_checks:
            check_func(dataset)
            
        # set transformer features

        # convert to accepted dtype and get attributes
        dataset = dataset.to_pandas()
        
        if self.check_mode: df = dataset.data
            
        cat_cols = get_columns_by_role(dataset, 'Category')
        num_cols = get_columns_by_role(dataset, 'Numeric')
        if self.verbose_mode: print('GroupByTransformer.__fit_new.cat_cols:', cat_cols)
        if self.verbose_mode: print('GroupByTransformer.__fit_new.num_cols:', num_cols)
        
        col_names = np.array(cat_cols + num_cols)
        col_values = dataset.data[col_names].to_numpy()

        feats = []
        for cat in cat_cols:
            cat_index = np.where(col_names == cat)[0][0]
            for num in num_cols:
                num_index = np.where(col_names == num)[0][0]
                
                feature = f'{self._fname_prefix}__{cat}_delta_mean_{num}'
                
                if self.check_mode: _dict_check = df[[cat, num]].groupby(cat)[num].mean().to_dict()
                _dict = {cat_current: np.nanmean(col_values[np.where(col_values[:, cat_index] == cat_current), num_index]) for cat_current in np.unique(col_values[:, cat_index])}
                
                if self.check_mode: assert np.array([np.allclose(_dict[k], _dict_check[k], equal_nan=True) for k in _dict]).all(), f'GroupByTransformer.__fit_new.not_equal.{cat}.{num}'
                
                self.dicts[feature] = {
                    'cat': cat, 
                    'cat_index': cat_index, 
                    'num': num, 
                    'num_index': num_index, 
                    'values': _dict, 
                    'kind': 'num_diff'
                }
                feats.append(feature)
                
        for cat1 in cat_cols:
            cat_index = np.where(col_names == cat1)[0][0]
            for cat2 in cat_cols:
                num_index = np.where(col_names == cat2)[0][0]
                if cat1 != cat2:
                    feature1 = f'{self._fname_prefix}__{cat1}_mode_{cat2}'
                    
                    if self.check_mode: _dict_check = df[[cat1, cat2]].groupby(cat1)[cat2].aggregate(self.__get_mode).to_dict()
                    _dict = {
                        cat_current: 
                            self.__get_mode(col_values[np.where(col_values[:, cat_index] == cat_current), num_index][0])
                        for cat_current in np.unique(col_values[:, cat_index])
                    }

                    if self.check_mode: assert np.array([np.allclose(_dict[k], _dict_check[k], equal_nan=True) for k in _dict]).all(), f'GroupByTransformer.__fit_new.not_equal.{cat1}.{cat2}'
                    
                    self.dicts[feature1] = {
                        'cat': cat1, 
                        'cat_index': cat_index, 
                        'num': cat2, 
                        'num_index': num_index, 
                        'values': _dict, 
                        'kind': 'cat_mode'
                    }
                    
                    feature2 = f'{self._fname_prefix}__{cat1}_is_mode_{cat2}'
                    self.dicts[feature2] = {
                        'cat': cat1, 
                        'cat_index': cat_index, 
                        'num': cat2, 
                        'num_index': num_index, 
                        'values': _dict, 
                        'kind': 'cat_ismode'
                    }
                    feats.extend([feature1, feature2])
            
        self._features = feats
        return self

    def transform(self, dataset):
        """transform"""

        # checks here
        super().transform(dataset)
        
        # convert to accepted dtype and get attributes
        if self.check_mode: df = dataset.data

        cat_cols = get_columns_by_role(dataset, 'Category')
        num_cols = get_columns_by_role(dataset, 'Numeric')
        if self.verbose_mode: print('GroupByTransformer.__transform_new.cat_cols:', cat_cols)
        if self.verbose_mode: print('GroupByTransformer.__transform_new.num_cols:', num_cols)

        col_names = np.array(cat_cols + num_cols)
        col_values = dataset.data[col_names].to_numpy()

        # transform
        roles = NumericRole()
        outputs = []
        
        if self.verbose_mode: 
            print(
#                 'GroupByTransformer.transform.self.dicts=', 
#                   *[(feat, (value['cat'], value['num'], value['kind'], value['values'], )) for feat, value in self.dicts.items()],
#                   sep='\n'
                 )
        
        for feat, value in self.dicts.items():
            cat, num = value['cat'], value['num']

            if value['kind'] == 'num_diff':
                if self.check_mode: new_arr_check = (df[num] - df[cat].map(value['values'])).values.reshape(-1, 1)
                new_arr = (col_values[:, value['num_index']] - [value['values'][k] if k in value['values'] else np.nan for k in col_values[:, value['cat_index']] ]).reshape(-1, 1)
                
                if self.check_mode: assert np.allclose(new_arr_check, new_arr, equal_nan=True), f'GroupByTransformer.__transform_new.num_diff.not_equal.{cat}.{num}'
    
            elif value['kind'] == 'cat_mode':
                if self.check_mode: new_arr_check = df[cat].map(value['values']).values.reshape(-1, 1)
                new_arr = np.array([value['values'][k] if k in value['values'] else np.nan for k in col_values[:, value['cat_index']] ]).reshape(-1, 1)
                
                if self.check_mode: assert np.allclose(new_arr_check, new_arr, equal_nan=True), f'GroupByTransformer.__transform_new.cat_mode.not_equal.{cat}.{num}'
                
            elif value['kind'] == 'cat_ismode':
                if self.check_mode: new_arr_check = (df[num] == df[cat].map(value['values'])).values.reshape(-1, 1)
                new_arr = (col_values[:, value['num_index']] == [value['values'][k] if k in value['values'] else np.nan for k in col_values[:, value['cat_index']] ]).reshape(-1, 1)
                
                if self.check_mode: assert np.allclose(new_arr_check, new_arr, equal_nan=True), f'GroupByTransformer.__transform_new.cat_ismode.not_equal.{cat}.{num}'

            output = dataset.empty().to_numpy()
            output.set_data(new_arr, [feat], roles)
            outputs.append(output)
            
        # create resulted        
        return dataset.empty().to_numpy().concat(outputs)


In [ ]:
class GroupByPipeline(FeaturesPipeline, TabularDataFeatures):
    def __init__(self, feats_imp = None, top_category: int = 3, top_numeric: int = 3, **kwargs):
        """
        """
        
        super().__init__(feats_imp=feats_imp)
        
        self.top_category = top_category
        self.top_numeric = top_numeric
        
        self.check_mode = kwargs['check_mode'] if 'check_mode' in kwargs else False
        self.verbose_mode = kwargs['verbose_mode'] if 'verbose_mode' in kwargs else False

    def create_pipeline(self, train):
        """create_pipeline"""
        
        if self.verbose_mode: print('GroupByPipeline.create_pipeline')

        transformer_list = []

        categories = get_columns_by_role(train, 'Category')
        if self.verbose_mode: print('GroupByPipeline.create_pipeline.categories:', categories)

        numerics = get_columns_by_role(train, 'Numeric')
        if self.verbose_mode: print('GroupByPipeline.create_pipeline.numerics:', numerics)

        cat_feats_to_select = []
        num_feats_to_select = []
        
        if len(categories) > self.top_category:
            cat_feats_to_select = self.get_top_categories(train, self.top_category)
        elif len(categories) > 0:
            cat_feats_to_select = categories
        if self.verbose_mode: print('GroupByPipeline.create_pipeline.cat_feats_to_select:', cat_feats_to_select)
            
        if len(numerics) > self.top_numeric:
            num_feats_to_select = self.get_top_numeric(train, self.top_numeric)
        elif len(numerics) > 0:
            num_feats_to_select = numerics        
        if self.verbose_mode: print('GroupByPipeline.create_pipeline.num_feats_to_select:', num_feats_to_select)

        if (len(cat_feats_to_select) > 0) and (len(num_feats_to_select) > 0):
            groupby_processing = SequentialTransformer([
                UnionTransformer([
                    SequentialTransformer([
                        ColumnsSelector(keys=categories),
                        LabelEncoder(subs=None, random_state=42),
#                         FillnaMedian(),
                    ]),
                    SequentialTransformer([
                        ColumnsSelector(keys=num_feats_to_select)]),
#                         FillnaMedian(),
                    ]),
#                 FillnaMedian(),
                
                GroupByTransformer(check_mode=self.check_mode, verbose_mode=self.verbose_mode),
            ])
            
            transformer_list.append(groupby_processing)
        else:
            raise ValueError('GroupByPipeline expects at least 1 categorial and 1 numeric features')                
            
        if self.verbose_mode: print('GroupByPipeline.create_pipeline.transformer_list:', transformer_list)

        return UnionTransformer(transformer_list)
    
    def get_top_numeric(self, train, top_n = 5):
        """get_top_numeric"""

        nums = get_columns_by_role(train, 'Numeric')
        if len(nums) == 0:
            return []

        df = pd.DataFrame({'importance': 0, 'cardinality': 0}, index=nums)
        # importance if defined
        if self.feats_imp is not None:
            feats_imp = pd.Series(self.feats_imp.get_features_score()).sort_values(ascending=False)
            df['importance'] = feats_imp[feats_imp.index.isin(nums)]
            df['importance'].fillna(-np.inf)

        # check for cardinality
        df['cardinality'] = -self.get_uniques_cnt(train, nums)
        # sort
        df = df.sort_values(by=['importance', 'cardinality'], ascending=[False, self.ascending_by_cardinality])
        # get top n
        top = list(df.index[:top_n])

        return top

In [ ]:
# roles = {'target': TARGET_NAME, 'category': ['profile_town', 'profile_district', 'engine_type', 'recommended_policy_type', 'car_color', 'car_type']}
roles = {'target': TARGET_NAME, 
         'drop': ['PassengerId', ],
        }

task = Task('binary', )
reader = PandasToPandasReader(task, cv=N_FOLDS, random_state=RANDOM_STATE, advanced_roles=False)

model0 = BoostLGBM(default_params={'learning_rate': 0.1, 'num_leaves': 64, 'seed': 42, 'num_threads': N_THREADS})
pie = ModelBasedImportanceEstimator()
selector = ImportanceCutoffSelector(LGBSimpleFeatures(), model0, pie, cutoff=-9999)

model1 = BoostLGBM(default_params={'learning_rate': 0.1, 'num_leaves': 64, 'seed': 42, 'num_threads': N_THREADS})
pie1 = NpPermutationImportanceEstimator()
selector1 = ImportanceCutoffSelector(EmptyFeaturePipeline(), model1, pie1, cutoff=-9999)

pipe = LGBSimplerFeatures(selector, top_category=4).append(GroupByPipeline(None, top_category=4, top_numeric=4, check_mode=True, verbose_mode=True))

model = BoostLGBM(default_params={'learning_rate': 0.05, 'num_leaves': 128, 'seed': 1, 'num_threads': N_THREADS})
pipeline = MLPipeline([(model),], pre_selection=selector, features_pipeline=pipe, post_selection=selector1)

automl = AutoML(reader, [
    [pipeline],
], skip_conn=False, verbose=1)

oof_pred = automl.fit_predict(train_data, roles = roles)

test_pred = automl.predict(test_data)

print('Check scores...')
print('OOF score: {}'.format(roc_auc_score(train_data[TARGET_NAME].values,
                                           oof_pred.data[:, 0])))
test_automl = roc_auc_score(test_data[TARGET_NAME].values, test_pred.data[:, 0])
print('TEST score: {}'.format(test_automl))
print('')
print(pie1.get_features_score())

In [ ]:
result.loc['all features', 'TEST score'] = np.round(test_automl, 4)
result

In [ ]:
# roles = {'target': TARGET_NAME, 'category': ['profile_town', 'profile_district', 'engine_type', 'recommended_policy_type', 'car_color', 'car_type']}
roles = {'target': TARGET_NAME, 
         'category': [],
         'drop': ['PassengerId', 'Name', 'Embarked', 'Sex', 'Ticket'],
        }

task = Task('binary', )
reader = PandasToPandasReader(task, cv=N_FOLDS, random_state=RANDOM_STATE, advanced_roles=False)

model0 = BoostLGBM(default_params={'learning_rate': 0.1, 'num_leaves': 64, 'seed': 42, 'num_threads': N_THREADS})
pie = ModelBasedImportanceEstimator()
selector = ImportanceCutoffSelector(LGBSimpleFeatures(), model0, pie, cutoff=-9999)

model1 = BoostLGBM(default_params={'learning_rate': 0.1, 'num_leaves': 64, 'seed': 42, 'num_threads': N_THREADS})
pie1 = NpPermutationImportanceEstimator()
selector1 = ImportanceCutoffSelector(EmptyFeaturePipeline(), model1, pie1, cutoff=0)

pipe = LGBSimplerFeatures(selector, top_category=4).append(GroupByPipeline(None, top_category=3, top_numeric=1))

model = BoostLGBM(default_params={'learning_rate': 0.05, 'num_leaves': 128, 'seed': 1, 'num_threads': N_THREADS})
pipeline = MLPipeline([(model),], pre_selection=selector, features_pipeline=pipe, post_selection=selector1)

automl = AutoML(reader, [
    [pipeline],
], skip_conn=False, verbose=0)

oof_pred = automl.fit_predict(train_data, roles = roles)

test_pred = automl.predict(test_data)

print('Check scores...')
print('OOF score: {}'.format(roc_auc_score(train_data[TARGET_NAME].values,
                                           oof_pred.data[:, 0])))
test_automl = roc_auc_score(test_data[TARGET_NAME].values, test_pred.data[:, 0])
print('TEST score: {}'.format(test_automl))
print('')
print(pie1.get_features_score())
print(len(model.features))

In [ ]:
result.loc['top imp, 1 iter imp, holdout', 'TEST score'] = np.round(test_automl, 4)
result

In [ ]:
class NpCrossValPermutationImportanceEstimator(ImportanceEstimator):
    """Cross validation Permuation importance based estimator.

    Importance calculate, using random permutation of items in single column for each feature.

    """

    def __init__(self, random_state: int = 42, n_permut: int = 1):
        super().__init__()
        self.random_state = random_state
        self.n_permut = n_permut


    def fit(self, train_valid = None,
            ml_algo = None,
            preds = None):
        
        permutation_importance = {}
        c = 0
        for n, (idx, train, valid) in enumerate(train_valid):
            normal_score = ml_algo.score(preds[idx])
            valid_data = valid.to_numpy()
            for it, col in enumerate(valid_data.features):
                save_col = deepcopy(valid_data[:, col])
                for j in range(self.n_permut):
                    permutation = np.random.RandomState(seed=self.random_state+j).permutation(valid_data.shape[0])
                    shuffled_col = valid_data[permutation, col]

                    valid_data[col] = shuffled_col

                    new_preds = ml_algo.predict_single_fold(ml_algo.models[n], valid_data)
                    new_preds = new_preds.reshape((new_preds.shape[0], -1))
                    preds_ds = valid_data.empty().to_numpy()
                    preds_ds = ml_algo._set_prediction(preds_ds, new_preds)
                    shuffled_score = ml_algo.score(preds_ds)
                    if col in permutation_importance:
                        permutation_importance[col] += (normal_score - shuffled_score)  / len(ml_algo.models) / self.n_permut
                    else:
                        permutation_importance[col] = (normal_score - shuffled_score)  / len(ml_algo.models) / self.n_permut

                    # Set normal column back to the dataset
                    valid_data[col] = save_col
                    c += 1
        
        self.raw_importances = pd.Series(permutation_importance).sort_values(ascending=False)

In [ ]:
# roles = {'target': TARGET_NAME, 'category': ['profile_town', 'profile_district', 'engine_type', 'recommended_policy_type', 'car_color', 'car_type']}
roles = {'target': TARGET_NAME, 
         'category': [],
         'drop': ['PassengerId', 'Name', 'Embarked', 'Sex', 'Ticket'],
        }

task = Task('binary', )
reader = PandasToPandasReader(task, cv=N_FOLDS, random_state=RANDOM_STATE, advanced_roles=False)

model0 = BoostLGBM(default_params={'learning_rate': 0.1, 'num_leaves': 64, 'seed': 42, 'num_threads': N_THREADS})
pie = ModelBasedImportanceEstimator()
selector = ImportanceCutoffSelector(LGBSimpleFeatures(), model0, pie, cutoff=-9999)

model1 = BoostLGBM(default_params={'learning_rate': 0.1, 'num_leaves': 64, 'seed': 42, 'num_threads': N_THREADS})
pie1 = NpCrossValPermutationImportanceEstimator(n_permut=20)
selector1 = ImportanceCutoffSelector(EmptyFeaturePipeline(), model1, pie1, cutoff=0)

pipe = LGBSimplerFeatures(selector, top_category=4).append(GroupByPipeline(None, top_category=3, top_numeric=1))

model = BoostLGBM(default_params={'learning_rate': 0.05, 'num_leaves': 128, 'seed': 1, 'num_threads': N_THREADS})
pipeline = MLPipeline([(model),], pre_selection=selector, features_pipeline=pipe, post_selection=selector1)

automl = AutoML(reader, [
    [pipeline],
], skip_conn=False, verbose=0)

oof_pred = automl.fit_predict(train_data, roles = roles)

test_pred = automl.predict(test_data)

print('Check scores...')
print('OOF score: {}'.format(roc_auc_score(train_data[TARGET_NAME].values,
                                           oof_pred.data[:, 0])))
test_automl = roc_auc_score(test_data[TARGET_NAME].values, test_pred.data[:, 0])
print('TEST score: {}'.format(test_automl))
print('')
print(pie1.get_features_score())
print(len(model.features))

In [ ]:
result.loc['top imp, 20 iter imp', 'TEST score'] = np.round(test_automl, 4)
result

In [ ]:
class PredefinedSelector(SelectionPipeline):

    def __init__(self, columns_to_select):
        super().__init__()
        self.columns_to_select = set(columns_to_select)

    def perform_selection(self, train_valid):

        assert len(self.columns_to_select) == len(self.columns_to_select.intersection(set(train_valid.features))), \
            'Columns to select not match with dataset features'
        self._selected_features = sorted(list(self.columns_to_select))

class IterativeFeatureSelector(SelectionPipeline):
    def __init__(self, feature_pipeline: FeaturesPipeline,
                 ml_algo = None,
                 imp_estimator = None,
                 fit_on_holdout = True,
                 feature_group_size = 5,
                 kind  = 'forward'
                 ):

        super().__init__(feature_pipeline, ml_algo, imp_estimator, fit_on_holdout)

        self.feature_group_size = feature_group_size
        self.kind = kind
        
    def _forward(self, train_valid = None):
        # Calculate or receive permutation importances scores
        imp = self.imp_estimator.get_features_score()

        features_to_check = [x for x in imp.index if x in set(train_valid.features)]

        # Perform iterative selection algo
        chunks = _create_chunks_from_list(features_to_check, self.feature_group_size)
        selected_feats = []
        cnt_without_update = 0
        cur_best_score = None

        for it, chunk in enumerate(chunks):
            selected_feats += chunk
            cs = PredefinedSelector(selected_feats)
            selected_cols_iterator = train_valid.apply_selector(cs)
            # Create copy of MLAlgo for iterative algo only
            ml_algo_for_iterative, preds = tune_and_fit_predict(deepcopy(self._empty_algo), self.tuner, selected_cols_iterator)

            cur_score = ml_algo_for_iterative.score(preds)

            if cur_best_score is None or cur_best_score < cur_score:
                cur_best_score = cur_score
                cnt_without_update = 0
            else:
                cnt_without_update += 1

                selected_feats = selected_feats[:-len(chunk)]

        imp = imp[imp.index.isin(selected_feats)]
        self.map_raw_feature_importances(imp)

        selected_feats = list(self.mapped_importances.index)
        self._selected_features = selected_feats
        
    def _backward(self, train_valid = None):
        imp = self.imp_estimator.get_features_score()
        #imp = imp[imp > 0]
        #print(imp)
        features_to_check = [x for x in imp.index if x in set(train_valid.features)]

        # Perform iterative selection algo
        chunks = _create_chunks_from_list(features_to_check[::-1], self.feature_group_size)
        selected_feats = sorted(features_to_check)
        cs = PredefinedSelector(selected_feats)
        selected_cols_iterator = train_valid.apply_selector(cs)
        ml_algo_for_iterative, preds = tune_and_fit_predict(deepcopy(self._empty_algo), self.tuner, selected_cols_iterator)
        cur_best_score = ml_algo_for_iterative.score(preds)
        cnt_without_update = 0
        for it, chunk in enumerate(chunks):
            if cnt_without_update >= 30:
                break
        
            selected_feats = sorted(list(set(selected_feats) -  set(chunk)))
            cs = PredefinedSelector(selected_feats)
            selected_cols_iterator = train_valid.apply_selector(cs)
            ml_algo_for_iterative, preds = tune_and_fit_predict(deepcopy(self._empty_algo), self.tuner, selected_cols_iterator)

            cur_score = ml_algo_for_iterative.score(preds)
            #print(it, cur_best_score, cur_score, chunk)
            if cur_best_score is None or cur_best_score < cur_score:
                cur_best_score = cur_score
                cnt_without_update = 0
            else:
                cnt_without_update += 1

                selected_feats = sorted(list(set(selected_feats) |  set(chunk)))
        imp = imp[imp.index.isin(selected_feats)]
        self.map_raw_feature_importances(imp)
        selected_feats = list(self.mapped_importances.index)
        self._selected_features = selected_feats

    def perform_selection(self, train_valid = None):
        if self.kind == 'backward':
            self._backward(train_valid)
        elif self.kind == 'forward':
            self._forward(train_valid)

In [ ]:
# roles = {'target': TARGET_NAME, 'category': ['profile_town', 'profile_district', 'engine_type', 'recommended_policy_type', 'car_color', 'car_type']}
roles = {'target': TARGET_NAME, 
         'category': [],
         'drop': ['PassengerId', 'Name', 'Embarked', 'Sex', 'Ticket'],
        }

task = Task('binary', )
reader = PandasToPandasReader(task, cv=N_FOLDS, random_state=RANDOM_STATE, advanced_roles=False)

model0 = BoostLGBM(default_params={'learning_rate': 0.1, 'num_leaves': 64, 'seed': 42, 'num_threads': N_THREADS})
pie = ModelBasedImportanceEstimator()
selector = ImportanceCutoffSelector(LGBSimpleFeatures(), model0, pie, cutoff=-9999)

model1 = BoostLGBM(default_params={'learning_rate': 0.1, 'num_leaves': 64, 'seed': 42, 'num_threads': N_THREADS})
pie1 = NpCrossValPermutationImportanceEstimator(n_permut=1)
selector1 = IterativeFeatureSelector(EmptyFeaturePipeline(), model1, pie1, feature_group_size=1, fit_on_holdout=True, kind='forward')

pipe = LGBSimplerFeatures(selector, top_category=4).append(GroupByPipeline(None, top_category=3, top_numeric=1))

model = BoostLGBM(default_params={'learning_rate': 0.05, 'num_leaves': 128, 'seed': 1, 'num_threads': N_THREADS})
pipeline = MLPipeline([(model),], pre_selection=selector, features_pipeline=pipe, post_selection=selector1)

automl = AutoML(reader, [
    [pipeline],
], skip_conn=False, verbose=0)

oof_pred = automl.fit_predict(train_data, roles = roles)

test_pred = automl.predict(test_data)

print('Check scores...')
print('OOF score: {}'.format(roc_auc_score(train_data[TARGET_NAME].values,
                                           oof_pred.data[:, 0])))
test_automl = roc_auc_score(test_data[TARGET_NAME].values, test_pred.data[:, 0])
print('TEST score: {}'.format(test_automl))

print(pie1.get_features_score())
print(len(model.features))

In [ ]:
result.loc['forward, 1 iter imp, holdout', 'TEST score'] = np.round(test_automl, 4)
result

In [ ]:
# roles = {'target': TARGET_NAME, 'category': ['profile_town', 'profile_district', 'engine_type', 'recommended_policy_type', 'car_color', 'car_type']}
roles = {'target': TARGET_NAME, 
         'category': [],
         'drop': ['PassengerId', 'Name', 'Embarked', 'Sex', 'Ticket'],
        }

task = Task('binary', )
reader = PandasToPandasReader(task, cv=N_FOLDS, random_state=RANDOM_STATE, advanced_roles=False)

model0 = BoostLGBM(default_params={'learning_rate': 0.1, 'num_leaves': 64, 'seed': 42, 'num_threads': N_THREADS})
pie = ModelBasedImportanceEstimator()
selector = ImportanceCutoffSelector(LGBSimpleFeatures(), model0, pie, cutoff=-9999)

model1 = BoostLGBM(default_params={'learning_rate': 0.1, 'num_leaves': 64, 'seed': 42, 'num_threads': N_THREADS})
pie1 = NpCrossValPermutationImportanceEstimator(n_permut=20)
selector1 = IterativeFeatureSelector(EmptyFeaturePipeline(), model1, pie1, feature_group_size=1, fit_on_holdout=False, kind='forward')

pipe = LGBSimplerFeatures(selector, top_category=4).append(GroupByPipeline(None, top_category=3, top_numeric=1))

model = BoostLGBM(default_params={'learning_rate': 0.05, 'num_leaves': 128, 'seed': 1, 'num_threads': N_THREADS})
pipeline = MLPipeline([(model),], pre_selection=selector, features_pipeline=pipe, post_selection=selector1)

automl = AutoML(reader, [
    [pipeline],
], skip_conn=False, verbose=0)

oof_pred = automl.fit_predict(train_data, roles = roles)

test_pred = automl.predict(test_data)

print('Check scores...')
print('OOF score: {}'.format(roc_auc_score(train_data[TARGET_NAME].values,
                                           oof_pred.data[:, 0])))
test_automl = roc_auc_score(test_data[TARGET_NAME].values, test_pred.data[:, 0])
print('TEST score: {}'.format(test_automl))

print(pie1.get_features_score())
print(len(model.features))

In [ ]:
result.loc['final_forward', 'TEST score'] = np.round(test_automl, 4)
result

In [ ]:
# roles = {'target': TARGET_NAME, 'category': ['profile_town', 'profile_district', 'engine_type', 'recommended_policy_type', 'car_color', 'car_type']}
roles = {'target': TARGET_NAME, 
         'category': [],
         'drop': ['PassengerId', 'Name', 'Embarked', 'Sex', 'Ticket'],
        }

task = Task('binary', )
reader = PandasToPandasReader(task, cv=N_FOLDS, random_state=RANDOM_STATE, advanced_roles=False)

model0 = BoostLGBM(default_params={'learning_rate': 0.1, 'num_leaves': 64, 'seed': 42, 'num_threads': N_THREADS})
pie = ModelBasedImportanceEstimator()
selector = ImportanceCutoffSelector(LGBSimpleFeatures(), model0, pie, cutoff=-9999)

model1 = BoostLGBM(default_params={'learning_rate': 0.1, 'num_leaves': 64, 'seed': 42, 'num_threads': N_THREADS})
pie1 = NpCrossValPermutationImportanceEstimator(n_permut=20)
selector1 = IterativeFeatureSelector(EmptyFeaturePipeline(), model1, pie1, feature_group_size=1, fit_on_holdout=False, kind='backward')

pipe = LGBSimplerFeatures(selector, top_category=4).append(GroupByPipeline(None, top_category=3, top_numeric=1))

model = BoostLGBM(default_params={'learning_rate': 0.05, 'num_leaves': 128, 'seed': 1, 'num_threads': N_THREADS})
pipeline = MLPipeline([(model),], pre_selection=selector, features_pipeline=pipe, post_selection=selector1)

automl = AutoML(reader, [
    [pipeline],
], skip_conn=False, verbose=0)

oof_pred = automl.fit_predict(train_data, roles = roles)

test_pred = automl.predict(test_data)

print('Check scores...')
print('OOF score: {}'.format(roc_auc_score(train_data[TARGET_NAME].values,
                                           oof_pred.data[:, 0])))
test_automl = roc_auc_score(test_data[TARGET_NAME].values, test_pred.data[:, 0])
print('TEST score: {}'.format(test_automl))

print(pie1.get_features_score())
print(len(model.features))

In [ ]:
result.loc['final_backward', 'TEST score'] = np.round(test_automl, 4)
result